<B>What Should we Do in this Data ? </B>
<Br>

1- Perform Analysis of the data related to this competition <Br>
2- Apply simple Forecasting models and use the same evaluation from the competition.<Br>
3- Apply ETS models (not all models explained, the one that you can see is suitable)<Br>
4- Apply the Arima model<Br>
5- Apply Prophet<Br>
6- Apply any machine learning of your choice<Br>
7- Choose the best among them and perform future forecasting on the test set and submit on Kaggle and add the score in your notebook sent to me,

<B>Dataset Description</B><br>
Historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

<B>Files</B><br>
train.csv - historical data including Sales<br>
test.csv - historical data excluding Sales<br>
store.csv - supplemental information about the stores
Data fields


<b>Columns</b> <br>
1 - **Id** - an Id that represents a (Store, Date) duple within the test set<br>
2 - **Store** - a unique Id for each store<br>
3 - **Sales** - the turnover for any given day (this is what you are predicting)<br>
4 - **Customers** - the number of customers on a given day<br>
5 - **Open** - an indicator for whether the store was open: 0 = closed, 1 = open<br>
6 - **StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools 1 are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None<br>
7 - **SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools<br>
8 - **StoreType** - differentiates between 4 different store models: a, b, c, d<br>
9 - **Assortment** - describes an assortment level: a = basic, b = extra, c = extended<br>
10 - **CompetitionDistance** - distance in meters to the nearest competitor store<br>
11 - **CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened<br>
12 - **Promo** - indicates whether a store is running a promo on that day<br>
13 - **Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating<br>
14 - **Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2<br>
15 - **PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

<h2><B>Import Libraries</B><h2>

In [23]:
import pandas as pd

<h2><B>Read Data</B><h2>

In [20]:
train = pd.read_csv("Data/train.csv",parse_dates=['Date'],index_col='Date',low_memory=False)   
store_df = pd.read_csv("Data/store.csv")    
test = pd.read_csv("Data/test.csv",parse_dates=['Date'],index_col='Date',low_memory=False)   

In [26]:
train.head(5)


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1
2015-07-31,2,5,6064,625,1,1,0,1
2015-07-31,3,5,8314,821,1,1,0,1
2015-07-31,4,5,13995,1498,1,1,0,1
2015-07-31,5,5,4822,559,1,1,0,1


In [ ]:
store_df.head(5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


<B>We need to merge the Stores Data to the Train Dataset to work on it </B>

<B>✅ 3. Why merge them?</B>
🔧 So that your training data has all the features:You want to predict Sales, so the model should understand things like:<Br>

1 - Does a StoreType a generally sell more than StoreType d? <Br>
2 - Do stores with shorter CompetitionDistance perform worse? <Br>
3 - Do Promo2 stores show more stable growth? <Br>

These insights only become learnable by your model if you merge that info into train.csv.

In [32]:
df_train = pd.merge(train , store_df, how='left', on=['Store'])
df_train.head(5)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
